In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

import sys
sys.path.insert(0,'/content/gdrive/My Drive/Food_VR/GSR_Preprocessing/W&S_D_Shimmer_GSR')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
pip install neurokit2

     |████████████████████████████████| 993kB 7.5MB/s 


In [ ]:
pip install biosppy==0.6.1

     |████████████████████████████████| 81kB 5.9MB/s 


In [ ]:
pip install mne

     |████████████████████████████████| 7.0MB 6.4MB/s 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time 
import re
import csv
import scipy.io
import biosppy
import mne
import neurokit2 as nk
import cvxopt as cv
import cvxopt.solvers
import ast
import os
import scipy.io
from sklearn.preprocessing import LabelEncoder
import time
import datetime
from datetime import datetime
import glob
from scipy.stats import zscore, norm
from neurokit2 import eda_phasic
from scipy.stats import linregress
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# this function is to convert the TimeStamp column (first column) from Unix Epoch time to standard datetime format
def TimeStamp_Conversion(ts):
  """
  we have a unix epoch time in milliseconds i.e, a string with a length of 13 charcters example:1.5789360034388428E12
  
  parameters:
  -----
  ts = Epoch timesatmp in milliseconds.

  Returns:
  -----
  Std_Unix = standard epoch timestamp in seconds.

  """

  float_Input = float(ts) # converting the string-type(1.5789360034388428E12) Unix Epoch to float-type(1578936003.4388428).

  # float input is divided by 1000 to convert the Unix epoch in milliseconds to seconds 
  Std_Unix = float_Input/1000

  datetime_Input = datetime.fromtimestamp(Std_Unix) 
  # datetime.fromtimestamp converts the unix epoch in seconds to datetime returns example:datetime.datetime(2020, 1, 13, 17, 20, 3, 438843)

  return Std_Unix

In [ ]:
def column_formatting(Timestamp_DF):
  """
  Column names of Timestamp annotation excel have column index attached to column name as we only need column name we are parsing column names.

  Parameters:
  -----
  Timestamp_DF = Input the DF after reading the timestamp annotationexcel file to  get list of column names['A1- ECG baseline start','B1- ECG baseline end',.....].

  Returns:
  -----
  Parsed_ColumnNames = list of parsed column names. ['ECG baseline start','ECG baseline end',....]

  """

  Parsed_colnames = ['Subject_ID'] ## Column with Participant ID is not named, so declaring first column as Subject_ID to an empty list

  for index in range(1,len(Timestamp_DF.columns)): ## Looping through the list of timestamp annotation columns list
    column = Timestamp_DF.columns[index][4:].lstrip() ## Drop first 3 indices of each column and strip space(" ") if present as left most
    Parsed_colnames.append(column) ## appending each column name after parsing

  return Parsed_colnames ## returns list fo parsed col names

In [ ]:
def Annotation_timestamp(timestamp_path, sheet_name):
  """
  This function is to change the column names of timestamp annotations table and convert timestamps from milliseconds to standart epoch format of seconds.

  Parameters:
  -----
  timestamp_path = path to the directory of file location
  sheet_name =  there are two sheets present in the file, we work on file named D.

  Results:
  -----
  VR_Timestamps_D = Clean dataframe of timestamp annotations table.

  """

  VR_TimeStamps_D = pd.read_excel(Timestamp_path, sheet_name) ## read timestamp annotation file
  Parsed_colnames = column_formatting(VR_TimeStamps_D) ## using the column_formatting function defined earlier parse columns
  VR_TimeStamps_D.columns = Parsed_colnames ## Change colnames of Dataframe using the parsed list of col names
  
  ## As timestamp is in string format and in milli seconds iterating through each column to change the timestamp to standard epoch format.
  for col in VR_TimeStamps_D.columns: 
    ## Using Timestamp_Conversion function and lambda fucntion to map the function to each row of the column.
    if col == 'Subject_ID':
      pass
    else:
      VR_TimeStamps_D[col] = VR_TimeStamps_D[col].map(lambda instance: TimeStamp_Conversion(instance)) 

  return VR_TimeStamps_D

In [ ]:
def Shimmers_csv2DF(path,filename):
  """
  This function is to read Shimmer data files and create a dataframe from tidy shimmers csv tables.

  Parameters:
  -----
  path = path to directory of shimmers file folder.

  filename = name of the file to be loaded.

  Results:
  -----
  Dataframe = organized and structured Shimmers Data.

  """

  with open(path + '/' + filename, 'r',) as file: # read the file
    reader = csv.reader(file)

    lists_eachrow = []
    for row in reader:
      lists_eachrow.append(row) # append each row in reader to a list

  del lists_eachrow[0] # del first row of list as it is only about \t delimiter used

  newlists = [] 
  # loop through the list of lists and split columnar values using the delimiter 
  for list_row in lists_eachrow:
    for row in list_row:
      newlists.append(list(row.split('\t')))
  # Extract subjectID from the file name for future use
  filename_parse = filename.replace("_", " ")
  Participant_ID = ast.literal_eval(re.findall(r'\b\d+\b', filename_parse)[0])
  
  # create dataframe from the list of columnar values 
  Dataframe = pd.DataFrame(newlists, columns = newlists[0])
  Dataframe = Dataframe.drop([0,1]) # drop columns 1 and 2 as are column names and units which we already have for new dataframe.
  Dataframe.reset_index(drop=True, inplace=True) # reset index

  return Dataframe, Participant_ID 

In [ ]:
def Sliding_Window_GSRFeatureExt(GSR_16Hz,Participant_ID, start_window, overlap, window_length):
  """
  This function is to create a dataframe consisting of statistical features extracted using sliding window technique on Phasic and Tonic components of GSR signal.

  Parameters:
  -----
  GSR_16Hz = This the Dataframe consisting of Phasic and Tonic components of GSR signal along with epoch timestamp of each instance.

  Participant_ID = Subject ID extracted in Shimmers_csv2DF function from the file name.
  
  start_window = 0, starting index of the sliding window
  
  overlap = 50, sliding window tech. with 50 overlap
  
  window_length = 100, length of each window (100 rows at once)

  Results:
  -----
  Dataframe consisting of statistical features like Mean Phasic&Tonic components, std Phasic&Tonic components, Count and Mean of Phasic&Tonic Peaks, 
  Min and Max, Slope, AUC of Phasic&Tonic components for each window of length 100 and overlap of 50.

 """
  # creating an array of phasic, tonic and timestamps from the GSR_16Hz dataframe, will be easy to calculate statistical features of numpy array.
  phasic  = np.array(GSR_16Hz['phasic']) 
  tonic  = np.array(GSR_16Hz['tonic'])
  timestamp = np.array(GSR_16Hz['TimeStamp'])
  dummy = list(range(0,len(GSR_16Hz))) # this is to use as secondary axis to calculate slope.

  end_window = window_length
  Start_timestamp = []
  Mean_timestamp = []
  end_timestamp = []
  Mean_phasic = []
  Mean_tonic = []
  Std_phasic = []
  Std_tonic = []
  CountPeak_phasic = []
  CountPeak_tonic = []
  MeanPeak_phasic = []
  MeanPeak_tonic = []
  Min_phasic = []
  Max_phasic = []
  Min_tonic = []
  Max_tonic = []
  Slope_tonic = []
  Slope_phasic = []
  AUC_tonic = []
  AUC_phasic = []
  while abs(end_window-window_length) <= len(GSR_16Hz): # loop conditon to carry out windowing and calculate features till the end 

    # creating start, mean and end timestamps helps to  make sure that each window completely falls in annotation period
    Start_timestamp.append(timestamp[start_window]) 
    Mean_timestamp.append(np.mean(timestamp[start_window:end_window]))
    try:
      end_timestamp.append(timestamp[end_window])
    except IndexError: 
      # index of last window might exceed the last timestamp of our data, if this happens it will use last timestamp of data as end timsatmp of last window
      end_timestamp.append(timestamp[-1])

    # using numpy to compute statistucal features like mean, std, count
    Mean_phasic.append(np.mean(phasic[start_window:end_window]))
    Mean_tonic.append(np.mean(tonic[start_window:end_window]))
    Std_phasic.append(np.std(phasic[start_window:end_window]))
    Std_tonic.append(np.std(tonic[start_window:end_window]))
    # scipy.signal.find_peaks exctracts the peaks present in teh signal
    CountPeak_phasic.append(scipy.signal.find_peaks(phasic[start_window:end_window])[0].size)
    CountPeak_tonic.append(scipy.signal.find_peaks(tonic[start_window:end_window])[0].size)
    MeanPeak_phasic.append(np.mean(scipy.signal.find_peaks(phasic[start_window:end_window])[0]))
    MeanPeak_tonic.append(np.mean(scipy.signal.find_peaks(tonic[start_window:end_window])[0]))
    Min_phasic.append(min(phasic[start_window:end_window]))
    Max_phasic.append(max(phasic[start_window:end_window]))
    Min_tonic.append(min(tonic[start_window:end_window]))
    Max_tonic.append(max(tonic[start_window:end_window]))
    # slope is computed using linregress module
    Slope_tonic.append(linregress(dummy[start_window:end_window],tonic[start_window:end_window])[0])
    Slope_phasic.append(linregress(dummy[start_window:end_window],phasic[start_window:end_window])[0])
    # Area under curve is calculated using sk learn metrics
    AUC_tonic.append(metrics.auc(dummy[start_window:end_window],tonic[start_window:end_window]))
    AUC_phasic.append(metrics.auc(dummy[start_window:end_window],phasic[start_window:end_window]))
      
    # to increment start and end for next window
    start_window += overlap
    end_window += overlap
  
  # create Data frame from the lists of values appended to each feature.
  Feature_DF = pd.DataFrame()
  Feature_DF['Start_timestamp'] = Start_timestamp
  Feature_DF['Mean_timestamp'] = Mean_timestamp
  Feature_DF['End_timestamp'] = end_timestamp
  Feature_DF['Mean_phasic'] = Mean_phasic
  Feature_DF['Mean_tonic'] = Mean_tonic
  Feature_DF['Std_phasic'] = Std_phasic
  Feature_DF['Std_tonic'] = Std_tonic
  Feature_DF['CountPeak_phasic'] = CountPeak_phasic
  Feature_DF['CountPeak_tonic'] = CountPeak_tonic
  Feature_DF['MeanPeak_phasic'] = MeanPeak_phasic
  Feature_DF['MeanPeak_tonic'] = MeanPeak_tonic
  Feature_DF['Min_phasic'] = Min_phasic
  Feature_DF['Max_phasic'] = Max_phasic
  Feature_DF['Min_tonic'] = Min_tonic
  Feature_DF['Max_tonic'] = Max_tonic
  Feature_DF['Slope_tonic'] = Slope_tonic
  Feature_DF['Slope_phasic'] = Slope_phasic
  Feature_DF['AUC_tonic'] = AUC_tonic
  Feature_DF['AUC_phasic'] = AUC_phasic
  Feature_DF['Subject_ID'] = Participant_ID

  return Feature_DF

In [ ]:
def Tonic_col(columns):
  '''
  --- This function is to extract tonic columns from the GSR features data
  '''
  tonic_col = []
  for x in columns:
    if 'tonic' in x:
      tonic_col.append(x)
  return tonic_col